Импорт библиотек, необходимых для вычислений

In [ ]:
import numpy as np
import math
from scipy import stats

In [ ]:
avg_x = {}; sigma = {}; SEM = {}; x_sl = {}; abs_x = {}; otn_x = {}

In [ ]:
for i in dict_lab:
    n = len(dict_lab[i])
    avg_x[i] = np.mean(dict_lab[i])
    sigma[i] = np.std(dict_lab[i], ddof=1)
    SEM[i] = np.std(dict_lab[i], ddof=1) / np.sqrt(np.size(dict_lab[i]))
    x_sl[i] = stats.t.ppf((1 + alpha)/2, n-1) * SEM[i]
    abs_x[i] = math.sqrt(x_sist[i] ** 2 + x_sl[i] ** 2)
    otn_x[i] = abs_x[i]/avg_x[i]

print(f"Найдены параметры оценки погрешности величин:")
print(f"Количество измерений n: {n}")
print("Средние значения измеренных величин <Xi>:", avg_x, sep='\n')
print("Средние квадратичные погрешности отдельных результатов измерений Sn:", sigma, sep='\n')
print("Средние квадратичные погрешности среднего арифметического S:", SEM, sep='\n')
print("Случайные погрешности ΔXсл:", x_sl, sep='\n')
print("Абсолютные погрешности ΔX:", abs_x, sep='\n')
print("Относительные погрешности δX:", otn_x, sep='\n')

In [ ]:
print(f"Найдем среднее значение {find_val}, используя средние значения измеренных величин:")

for i in dict_lab:
    print(f"{i}:{avg_x[i]}")
    globals()[i] = avg_x[i]

avg_f = func()

print(f"Среднее значение {find_val} = {avg_f}")

In [ ]:
dx = []
import sympy as sym
for i in dict_lab:
    globals()[i] = sym.Symbol(i)
    dx.append(sym.Symbol(i))

f = func()

In [ ]:
derivative_f = []
dfdx = []
unpac_avg = []

for i in val_names:
    globals()[i] = avg_x[i]
    unpac_avg.append(avg_x[i])

unpac_avg = tuple(unpac_avg)

for i in range(len(val_names)):
    df = f.diff(dx[i])
    derivative_f.append(df)
    f_i = sym.lambdify([j for j in dx], df)
    dfdx.append(f_i(*unpac_avg))

print(f"Для функции f = {f} найдем значения производных df/dxi:")
for i, v in enumerate(val_names):
    print(f"df/d{v}: {dfdx[i]}")

In [ ]:
print(f"Подставим df/dxi и абсолютные погрешности прямых измерений Δxi в формулу для нахождения абсолютной погрешности косвенного измерения")

abs_f = 0
for i, v in enumerate(dict_lab):
    abs_f += (dfdx[i] * abs_x[v]) ** 2

abs_f = math.sqrt(abs_f)
otn_f = abs_f/avg_f

print(f"Абсолютная погрешность Δ{find_val} = {abs_f}")
print(f"Найдем относительную погрешность функции f, зная ее среднее значение и абсолютную погрешность")
print(f"Относительная погрешность δ{find_val} = {otn_f} или {round(otn_f * 100, 2)}%")

In [ ]:
print(f"Добавляем найденные значения в БД {result_table}")

connection = create_connection("postgres", "postgres", "1234", "localhost", "5432")

for i in dict_lab:
    update = f"""
UPDATE {result_table}
SET
avg_x = '{avg_x[i]}',
abs_x = '{abs_x[i]}',
otn_x = '{otn_x[i]}',
s = '{SEM[i]}'
WHERE
value = '{i}'
"""
    update_table = execute_query(connection, update)
    print(f'В БД {result_table} добавлены найденные значения параметров оценки погрешности величины {i}')

update = f"""
UPDATE {result_table}
SET
avg_x = '{avg_f}',
abs_x = '{abs_f}',
otn_x = '{otn_f}'
WHERE
value = '{find_val}'
"""

update_table = execute_query(connection, update)
print(f'В БД {result_table} добавлены найденные значения параметров оценки погрешности косвенной величины {find_val}')